In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('Dataset_A_loan.csv')

In [3]:
print(data.dtypes)

person_age                        float64
person_gender                      object
person_education                   object
person_income                     float64
person_emp_exp                      int64
person_home_ownership              object
loan_amnt                         float64
loan_intent                        object
loan_int_rate                     float64
loan_percent_income               float64
cb_person_cred_hist_length        float64
credit_score                        int64
previous_loan_defaults_on_file     object
loan_status                         int64
dtype: object


In [4]:
data.head(10)

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
5,21.0,female,High School,NaN,0,OWN,2500.0,VENTURE,7.14,0.19,2.0,532,No,1
6,26.0,female,Bachelor,93471.0,1,RENT,35000.0,EDUCATION,12.42,0.37,3.0,701,No,1
7,24.0,female,High School,95550.0,5,RENT,35000.0,MEDICAL,11.11,0.37,4.0,585,No,1
8,24.0,female,Associate,100684.0,3,RENT,35000.0,PERSONAL,8.90,0.35,2.0,544,No,1
9,21.0,female,High School,12739.0,0,OWN,1600.0,VENTURE,14.74,0.13,3.0,640,No,1


In [5]:
data.info

<bound method DataFrame.info of        person_age person_gender person_education  person_income  \
0            22.0        female           Master        71948.0   
1            21.0        female      High School        12282.0   
2            25.0        female      High School        12438.0   
3            23.0        female         Bachelor        79753.0   
4            24.0          male           Master        66135.0   
...           ...           ...              ...            ...   
44995        27.0          male        Associate        47971.0   
44996        37.0        female        Associate        65800.0   
44997        33.0          male        Associate        56942.0   
44998        29.0          male         Bachelor        33164.0   
44999        24.0          male      High School        51609.0   

       person_emp_exp person_home_ownership  loan_amnt        loan_intent  \
0                   0                  RENT    35000.0           PERSONAL   
1        

In [6]:
print(data.describe())

         person_age  person_income  person_emp_exp     loan_amnt  \
count  45000.000000   4.275000e+04    45000.000000  45000.000000   
mean      27.764178   8.018223e+04        5.410333   9583.157556   
std        6.045108   7.331158e+04        6.063532   6314.886691   
min       20.000000   8.000000e+03        0.000000    500.000000   
25%       24.000000   4.726725e+04        1.000000   5000.000000   
50%       26.000000   6.705500e+04        4.000000   8000.000000   
75%       30.000000   9.582125e+04        8.000000  12237.250000   
max      144.000000   5.556399e+06      125.000000  35000.000000   

       loan_int_rate  loan_percent_income  cb_person_cred_hist_length  \
count   45000.000000         45000.000000                45000.000000   
mean       11.006606             0.139725                    5.867489   
std         2.978808             0.087212                    3.879702   
min         5.420000             0.000000                    2.000000   
25%         8.590000  

#### cek total missing value dari tiap kolom

In [8]:
misval = data.isnull().sum()
misval = misval[misval > 0]  
print(misval)

person_income    2250
dtype: int64


In [9]:
data['person_income'].skew()


25.60673912825341

karena hasilnya jauh dari 0 artinya skewed yang berarti lebih baik diii oleh median karena lebih robust(tahan terhadap outlier)

#### mengisi missing value dengan median

In [12]:
median_income = data['person_income'].median()
data['person_income'] = data['person_income'].fillna(median_income)

#### cek unique value untuk normalisasi data

In [14]:
for col in data.columns:
    unique_values = data[col].unique()
    print(f"{col}: {len(unique_values)} : {unique_values[:6]}")  

person_age: 60 : [22. 21. 25. 23. 24. 26.]
person_gender: 4 : ['female' 'male' 'Male' 'fe male']
person_education: 5 : ['Master' 'High School' 'Bachelor' 'Associate' 'Doctorate']
person_income: 32637 : [71948. 12282. 12438. 79753. 66135. 67055.]
person_emp_exp: 63 : [0 3 1 5 4 2]
person_home_ownership: 4 : ['RENT' 'OWN' 'MORTGAGE' 'OTHER']
loan_amnt: 4483 : [35000.  1000.  5500.  2500.  1600.  4500.]
loan_intent: 6 : ['PERSONAL' 'EDUCATION' 'MEDICAL' 'VENTURE' 'HOMEIMPROVEMENT'
 'DEBTCONSOLIDATION']
loan_int_rate: 1302 : [16.02 11.14 12.87 15.23 14.27  7.14]
loan_percent_income: 64 : [0.49 0.08 0.44 0.53 0.19 0.37]
cb_person_cred_hist_length: 29 : [3. 2. 4. 8. 7. 6.]
credit_score: 340 : [561 504 635 675 586 532]
previous_loan_defaults_on_file: 2 : ['No' 'Yes']
loan_status: 2 : [1 0]


#### normalisasi gender

In [16]:
data['person_gender'] = (
    data['person_gender']
    .str.lower()
    .str.replace(" ", "", regex=False)
    .map({'male': 'male', 'female': 'female'})
)

In [17]:
data['person_gender'].unique()

array(['female', 'male'], dtype=object)

#### binary encode untuk gender

In [19]:
gender_map = {'male': 1, 'female': 0}
data['person_gender'] = data['person_gender'].map(gender_map)

In [20]:
data['person_gender'].unique()

array([0, 1], dtype=int64)

#### ordinal encoding unntuk education

In [22]:
education_order = {
    'High School': 0,
    'Associate': 1,
    'Bachelor': 2,
    'Master': 3,
    'Doctorate': 4
}
data['person_education'] = data['person_education'].map(education_order)

In [23]:
data['person_education'].unique()

array([3, 0, 2, 1, 4], dtype=int64)

#### binary encoding untuk previous loan default 

In [25]:
loan_default_map = {'No': 0, 'Yes': 1}
data['previous_loan_defaults_on_file'] = data['previous_loan_defaults_on_file'].map(loan_default_map)

In [26]:
data['previous_loan_defaults_on_file'].unique()

array([0, 1], dtype=int64)

#### one hot encoding untuk home ownership

In [28]:
data = pd.get_dummies(data, columns=['person_home_ownership'], prefix='home', drop_first=False)

#### one hot encoding untuk loan intent

In [30]:
data = pd.get_dummies(data, columns=['loan_intent'], prefix='intent', drop_first=False)

#### ubah dari bool ke int

In [32]:
dummy_cols = [
    'home_MORTGAGE', 'home_OTHER', 'home_OWN', 'home_RENT',
    'intent_DEBTCONSOLIDATION', 'intent_EDUCATION', 'intent_HOMEIMPROVEMENT',
    'intent_MEDICAL', 'intent_PERSONAL', 'intent_VENTURE'
]

data[dummy_cols] = data[dummy_cols].astype(int)

In [33]:
for col in data.columns:
    unique_values = data[col].unique()
    print(f"{col}: {len(unique_values)} : {unique_values[:5]}") 

person_age: 60 : [22. 21. 25. 23. 24.]
person_gender: 2 : [0 1]
person_education: 5 : [3 0 2 1 4]
person_income: 32637 : [71948. 12282. 12438. 79753. 66135.]
person_emp_exp: 63 : [0 3 1 5 4]
loan_amnt: 4483 : [35000.  1000.  5500.  2500.  1600.]
loan_int_rate: 1302 : [16.02 11.14 12.87 15.23 14.27]
loan_percent_income: 64 : [0.49 0.08 0.44 0.53 0.19]
cb_person_cred_hist_length: 29 : [3. 2. 4. 8. 7.]
credit_score: 340 : [561 504 635 675 586]
previous_loan_defaults_on_file: 2 : [0 1]
loan_status: 2 : [1 0]
home_MORTGAGE: 2 : [0 1]
home_OTHER: 2 : [0 1]
home_OWN: 2 : [0 1]
home_RENT: 2 : [1 0]
intent_DEBTCONSOLIDATION: 2 : [0 1]
intent_EDUCATION: 2 : [0 1]
intent_HOMEIMPROVEMENT: 2 : [0 1]
intent_MEDICAL: 2 : [0 1]
intent_PERSONAL: 2 : [1 0]
intent_VENTURE: 2 : [0 1]


#### fitur(x) dan target (y)

In [35]:
X = data.drop(columns=['loan_status'])
y = data['loan_status']

#### split dataset ke training dan testing

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### scalling fitur numerik

In [39]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### random forest training

In [41]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

#### prediksi dan evaluasi random forest

In [43]:
rf_preds = rf_model.predict(X_test_scaled)
rf_accuracy = accuracy_score(y_test, rf_preds)

#### training dengan xgboost

In [45]:
xgb_model = XGBClassifier(random_state=42, n_estimators=50, eval_metric='logloss')
xgb_model.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=50,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

#### prediksi dan evaluasi xgboost

In [47]:
xgb_preds = xgb_model.predict(X_test_scaled)
xgb_accuracy = accuracy_score(y_test, xgb_preds)

#### performance dari kedua model

In [49]:
print(f"Random Forest Accuracy: {rf_accuracy}")
print(f"XGBoost Accuracy: {xgb_accuracy}")

Random Forest Accuracy: 0.9296666666666666
XGBoost Accuracy: 0.9318888888888889


#### menyimpan model terbaik (XGBoost) dalam pickle

In [51]:
with open('best_xgb_model.pkl', 'wb') as model_file:
    pickle.dump(xgb_model, model_file)

#### pickle untuk scalling

In [53]:
scaler = StandardScaler()

with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)